In [1]:
import os
import re
import json
import pandas as pd
from docx import Document
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import joblib

def extract_json_from_docx(file_path):
    """
    Extracts JSON string from a .docx file.
    Assumes that the JSON content is enclosed within [ ] brackets.
    """
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

    full_text = []
    json_started = False

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue  # Skip empty paragraphs

        # Detect the start of JSON array
        if text.startswith('['):
            json_started = True

        if json_started:
            full_text.append(text)
            # Detect the end of JSON array
            if text.endswith(']'):
                break

    json_str = '\n'.join(full_text)

    # Optional: Clean up the JSON string using regex if necessary
    # For example, remove unwanted characters or fix formatting issues
    json_str = re.sub(r'(?<!\\)"', r'"', json_str)  # Replace unescaped quotes if necessary

    return json_str

def parse_json_to_dataframe(json_str):
    """
    Parses a JSON string to a pandas DataFrame.
    Handles both single JSON array and multiple JSON objects.
    """
    if not json_str:
        return None

    try:
        # Attempt to load the JSON string as a list
        data = json.loads(json_str)
        if isinstance(data, list):
            df = pd.json_normalize(data, sep='.')
            return df
        else:
            # If not a list, wrap it into a list
            df = pd.json_normalize([data], sep='.')
            return df
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return None

def process_docx_files(directory_path):
    """
    Processes all .docx files in the specified directory.
    Returns a combined pandas DataFrame.
    """
    all_dfs = []
    
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.docx'):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")
            json_str = extract_json_from_docx(file_path)
            df = parse_json_to_dataframe(json_str)
            if df is not None:
                all_dfs.append(df)
            else:
                print(f"Failed to parse JSON in file: {filename}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        return combined_df
    else:
        print("No DataFrames to concatenate.")
        return pd.DataFrame()

def encode_airline_names(df, top_n=10):
    """
    Encodes 'airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    if 'airline.name' in df.columns:
        top_airlines = df['airline.name'].value_counts().nlargest(top_n).index
        df['airline.name'] = df['airline.name'].apply(lambda x: x if x in top_airlines else 'Other')
        df = pd.get_dummies(df, columns=['airline.name'], prefix='airline', drop_first=True)
        print(f"One-Hot Encoded 'airline.name' with top {top_n} categories.")
    return df

def encode_codeshare_airline_names(df, top_n=10):
    """
    Encodes 'codeshared.airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    # Adjust the column name based on actual DataFrame columns
    possible_column_names = ['codeshared.airline.name', 'codeshare.airline.name', 'codeshare_airline.name']
    column_found = False
    for col in possible_column_names:
        if col in df.columns:
            top_airlines = df[col].value_counts().nlargest(top_n).index
            df[col] = df[col].apply(lambda x: x if x in top_airlines else 'Other')
            df = pd.get_dummies(df, columns=[col], prefix='codeshare_airline', drop_first=True)
            print(f"One-Hot Encoded '{col}' with top {top_n} categories.")
            column_found = True
            break
    if not column_found:
        print("No 'codeshared.airline.name' column found to encode.")
    return df

def encode_categorical_variables(df):
    """
    Encodes categorical variables into numerical formats.
    """
    # Initialize LabelEncoder
    le = LabelEncoder()

    # Encode 'status' column (binary or multi-class classification)
    if 'status' in df.columns:
        df['status_encoded'] = le.fit_transform(df['status'])
        print("Encoded 'status' column.")

    # One-Hot Encode 'departure.day_of_week' if exists
    if 'departure.day_of_week' in df.columns:
        df = pd.get_dummies(df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # One-Hot Encode 'airline.name' with top categories
    df = encode_airline_names(df, top_n=10)

    return df

def encode_all_categorical_variables(df, top_n=10):
    """
    Encodes all relevant categorical variables into numerical formats.
    """
    df = encode_categorical_variables(df)
    df = encode_codeshare_airline_names(df, top_n=top_n)
    return df

def drop_high_missing_columns(df, threshold=50):
    """
    Drops columns from the DataFrame where the percentage of missing values exceeds the threshold.
    """
    missing_percentage = df.isnull().mean() * 100
    cols_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped columns with >{threshold}% missing values: {cols_to_drop}")
    return df

def impute_datetime_columns(df, datetime_cols):
    """
    Converts specified columns to datetime and imputes missing values based on a logical hierarchy.
    """
    for col in datetime_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
            print(f"Converted '{col}' to datetime.")

    # Impute 'departure.estimatedTime' and 'arrival.estimatedTime' with median
    for col in ['departure.estimatedTime', 'arrival.estimatedTime']:
        if col in df.columns and df[col].isnull().sum() > 0:
            median_time = df[col].median()
            df[col].fillna(median_time, inplace=True)
            print(f"Imputed missing '{col}' with median time {median_time}.")

    # Now, impute 'departure.actualTime' with 'departure.estimatedTime'
    if 'departure.actualTime' in df.columns and 'departure.estimatedTime' in df.columns:
        df['departure.actualTime'].fillna(df['departure.estimatedTime'], inplace=True)
        print("Imputed missing 'departure.actualTime' with 'departure.estimatedTime'.")

    # Similarly, impute 'arrival.actualTime' with 'arrival.estimatedTime'
    if 'arrival.actualTime' in df.columns and 'arrival.estimatedTime' in df.columns:
        df['arrival.actualTime'].fillna(df['arrival.estimatedTime'], inplace=True)
        print("Imputed missing 'arrival.actualTime' with 'arrival.estimatedTime'.")

    return df

def impute_remaining_columns(df):
    """
    Imputes remaining numerical columns with median and categorical columns with 'Unknown'.
    """
    # Identify numerical and categorical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    # Impute numerical columns with median
    for col in numerical_cols:
        if df[col].isnull().sum() > 0:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"Imputed missing values in numerical column '{col}' with median value {median_value}.")

    # Impute categorical columns with 'Unknown'
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna('Unknown', inplace=True)
            print(f"Imputed missing values in categorical column '{col}' with 'Unknown'.")

    return df

def remove_low_variance_features(df, threshold=0.0):
    """
    Removes numerical features with variance below the specified threshold.
    Non-numerical columns are retained without modification.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - threshold (float): The variance threshold.
    
    Returns:
    - pd.DataFrame: The DataFrame with low variance numerical features removed.
    """
    # Select numerical columns (int64, float64, uint8)
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()

    if not numeric_cols:
        print("No numerical columns to apply VarianceThreshold.")
        return df

    # Initialize VarianceThreshold
    selector = VarianceThreshold(threshold=threshold)

    # Fit the selector on numerical data
    try:
        selector.fit(df[numeric_cols])
    except ValueError as e:
        print(f"VarianceThreshold failed: {e}")
        return df

    # Get the columns to keep
    features_to_keep = [col for col, keep in zip(numeric_cols, selector.get_support()) if keep]

    # Retain selected numerical features and all non-numerical features
    non_numeric_cols = df.columns.difference(numeric_cols)
    df_filtered = pd.concat([df[non_numeric_cols], df[features_to_keep]], axis=1)

    print(f"Removed low variance features. Remaining features: {len(df_filtered.columns)}")
    
    return df_filtered

def load_weather_data(weather_directory, num_files=13):
    """
    Loads and concatenates weather data from multiple Excel files.
    
    Parameters:
    - weather_directory (str): Path to the directory containing weather Excel files.
    - num_files (int): Number of Excel files to load (e.g., first 13 files).
    
    Returns:
    - pd.DataFrame: Combined weather data.
    """
    weather_dfs = []
    for i in range(1, num_files + 1):
        file_name = f"{i}.xlsx"
        file_path = os.path.join(weather_directory, file_name)
        if os.path.exists(file_path):
            try:
                df_weather = pd.read_excel(file_path)
                print(f"Loaded weather data from {file_name}")
                weather_dfs.append(df_weather)
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
        else:
            print(f"Weather file {file_name} does not exist in {weather_directory}. Skipping.")
    
    if weather_dfs:
        combined_weather_df = pd.concat(weather_dfs, ignore_index=True)
        print(f"Combined weather DataFrame shape: {combined_weather_df.shape}")
        return combined_weather_df
    else:
        print("No weather data files loaded.")
        return pd.DataFrame()

def reshape_weather_data(weather_df, year=2023):
    """
    Reshapes the weather data from wide to long format.

    Assumes that:
    - The first column is 'Time' containing metric names (e.g., Max, Avg, Min)
    - Subsequent columns are days (e.g., Jul 1, Jul 2, ..., Jul 31)

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data.
    - year (int): Year for the dates.

    Returns:
    - pd.DataFrame: Reshaped weather data with one row per day and separate columns for each metric.
    """
    # Check if 'Time' column exists
    if 'Time' not in weather_df.columns:
        print("No 'Time' column found in weather data.")
        return weather_df

    # Rename 'Time' to 'Metric' for clarity
    weather_df.rename(columns={'Time': 'Metric'}, inplace=True)

    # Melt the DataFrame to long format
    weather_long_df = weather_df.melt(id_vars=['Metric'], var_name='Day', value_name='Value')

    # Parse the 'Day' column to create a 'Date' column
    # Assuming 'Day' is in the format 'Jul 1', 'Jul 2', etc.
    weather_long_df['Date'] = pd.to_datetime(
        weather_long_df['Day'] + f' {year}',
        format='%b %d %Y',
        errors='coerce'
    )

    # Drop rows with invalid dates
    initial_shape = weather_long_df.shape
    weather_long_df.dropna(subset=['Date'], inplace=True)
    final_shape = weather_long_df.shape
    print(f"Dropped {initial_shape[0] - final_shape[0]} rows with invalid 'Date'.")

    # Pivot the DataFrame to have metrics as separate columns
    weather_pivot = weather_long_df.pivot_table(
        index='Date',
        columns='Metric',
        values='Value'
    ).reset_index()

    # Rename 'Total' to 'Precipitation (in)' if applicable
    if 'Total' in weather_pivot.columns:
        weather_pivot.rename(columns={'Total': 'Precipitation (in)'}, inplace=True)

    # Convert metric columns to numeric types
    for col in weather_pivot.columns:
        if col != 'Date':
            weather_pivot[col] = pd.to_numeric(weather_pivot[col], errors='coerce')

    print(f"Reshaped weather DataFrame shape: {weather_pivot.shape}")
    return weather_pivot

def preprocess_weather_data(weather_df, year=2023):
    """
    Preprocesses the weather data DataFrame.

    - Reshapes the data from wide to long format.
    - Parses 'Date' column to datetime.
    - Handles missing values if any.

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data.
    - year (int): Year for the dates.

    Returns:
    - pd.DataFrame: Preprocessed weather data.
    """
    # Reshape the weather data
    weather_pivot = reshape_weather_data(weather_df, year=year)

    # Verify the reshaped data
    if weather_pivot.empty:
        print("Reshaping resulted in an empty DataFrame.")
        return weather_pivot

    return weather_pivot

def merge_flight_weather(flight_df, weather_df):
    """
    Merges flight data with weather data based on departure date.
    
    Parameters:
    - flight_df (pd.DataFrame): Cleaned flight data with 'departure.scheduledTime'.
    - weather_df (pd.DataFrame): Cleaned and reshaped weather data with 'Date'.
    
    Returns:
    - pd.DataFrame: Merged DataFrame.
    """
    # Ensure 'departure.scheduledTime' is datetime
    if not pd.api.types.is_datetime64_any_dtype(flight_df['departure.scheduledTime']):
        flight_df['departure.scheduledTime'] = pd.to_datetime(flight_df['departure.scheduledTime'], errors='coerce')
        print("Converted 'departure.scheduledTime' to datetime.")
    
    # Create a 'Departure Date' column (date only, no time)
    flight_df['Departure Date'] = flight_df['departure.scheduledTime'].dt.date
    weather_df['Date'] = weather_df['Date'].dt.date  # Ensure 'Date' is date only

    # Merge on 'Departure Date' and 'Date'
    merged_df = pd.merge(
        flight_df,
        weather_df,
        how='left',
        left_on='Departure Date',
        right_on='Date',
        suffixes=('', '_weather')
    )

    # Drop the redundant 'Date' column from weather data
    merged_df.drop(columns=['Date'], inplace=True)

    print(f"Merged flight data with weather data. Merged DataFrame shape: {merged_df.shape}")
    return merged_df

def encode_and_save_label_encoder(df, column, encoder_filename):
    """
    Encodes a categorical column using LabelEncoder and saves the encoder.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the column.
    - column (str): The name of the column to encode.
    - encoder_filename (str): The filename to save the encoder.
    
    Returns:
    - pd.DataFrame: The DataFrame with the encoded column.
    """
    if column in df.columns:
        le = LabelEncoder()
        df[f'{column}_encoded'] = le.fit_transform(df[column])
        joblib.dump(le, encoder_filename)
        print(f"Encoded '{column}' column and saved LabelEncoder as '{encoder_filename}'.")
    return df

def main():
    # Specify the directories containing your data
    flight_data_directory = 'ML-Proj-Dataset/Train/'    # Replace with your actual flight data path
    weather_data_directory = 'ML-Proj-Dataset/Weather/'         # Replace with your actual weather data path

    # Check if the directories exist
    if not os.path.isdir(flight_data_directory):
        print(f"The directory '{flight_data_directory}' does not exist. Please check the path.")
        return
    if not os.path.isdir(weather_data_directory):
        print(f"The directory '{weather_data_directory}' does not exist. Please check the path.")
        return

    # Process the .docx flight data files and get the combined DataFrame
    flight_df = process_docx_files(flight_data_directory)

    if flight_df.empty:
        print("No flight data to process.")
        return

    print(f"\nInitial Flight DataFrame shape: {flight_df.shape}")

    # Define datetime columns in flight data
    flight_datetime_cols = [
        'departure.scheduledTime',
        'departure.estimatedTime',
        'departure.actualTime',
        'departure.estimatedRunway',
        'departure.actualRunway',
        'arrival.scheduledTime',
        'arrival.estimatedTime',
        'arrival.actualTime',
        'arrival.estimatedRunway',
        'arrival.actualRunway'
    ]

    # Drop columns with >50% missing values
    flight_df = drop_high_missing_columns(flight_df, threshold=50)

    # Convert and impute datetime columns
    flight_df = impute_datetime_columns(flight_df, flight_datetime_cols)

    # Impute remaining numerical and categorical columns
    flight_df = impute_remaining_columns(flight_df)

    # Encode categorical variables with optimized methods
    flight_df = encode_all_categorical_variables(flight_df, top_n=10)

    # Remove low variance numerical features
    flight_df = remove_low_variance_features(flight_df, threshold=0.01)  # Adjust threshold as needed

    # Calculate departure delay if possible
    if 'departure.actualTime' in flight_df.columns and 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.delay_minutes'] = (flight_df['departure.actualTime'] - flight_df['departure.scheduledTime']).dt.total_seconds() / 60
        print("Calculated 'departure.delay_minutes'.")

    # Extract temporal features if needed
    if 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.day_of_week'] = flight_df['departure.scheduledTime'].dt.day_name()
        flight_df['departure.hour_of_day'] = flight_df['departure.scheduledTime'].dt.hour
        flight_df['departure.month'] = flight_df['departure.scheduledTime'].dt.month
        print("Extracted temporal features.")

        # One-Hot Encode temporal features
        flight_df = pd.get_dummies(flight_df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # Remove low variance numerical features again after adding temporal features
    flight_df = remove_low_variance_features(flight_df, threshold=0.0)  # Remove features with zero variance

    # Impute 'departure.delay_minutes' with median if there are still missing values
    if 'departure.delay_minutes' in flight_df.columns and flight_df['departure.delay_minutes'].isnull().sum() > 0:
        median_delay = flight_df['departure.delay_minutes'].median()
        flight_df['departure.delay_minutes'].fillna(median_delay, inplace=True)
        print(f"Imputed missing 'departure.delay_minutes' with median value {median_delay}.")

    # Load and preprocess weather data
    weather_df = load_weather_data(weather_data_directory, num_files=13)
    if weather_df.empty:
        print("No weather data available for merging.")
    else:
        weather_df = preprocess_weather_data(weather_df, year=2023)  # Adjust year if necessary

        # Merge flight data with weather data
        flight_df = merge_flight_weather(flight_df, weather_df)

        # Handle any remaining missing weather data by imputing with median
        weather_metrics = ['Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)', 
                           'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)']
        
        for metric in weather_metrics:
            if metric in flight_df.columns:
                if flight_df[metric].isnull().sum() > 0:
                    median_value = flight_df[metric].median()
                    flight_df[metric].fillna(median_value, inplace=True)
                    print(f"Imputed missing '{metric}' with median value {median_value}.")

    # Save LabelEncoder for 'status'
    if 'status_encoded' in flight_df.columns:
        le_status = LabelEncoder()
        le_status.fit(flight_df['status'])
        joblib.dump(le_status, 'label_encoder_status.pkl')
        print("Saved LabelEncoder for 'status' column as 'label_encoder_status.pkl'.")

    # Save the final cleaned and merged DataFrame
    flight_df.to_csv('final_cleaned_flight_data_with_weather.csv', index=False)
    print("\nFinal cleaned and merged DataFrame saved as 'final_cleaned_flight_data_with_weather.csv'.")

    # Display final DataFrame information
    print("\nFinal DataFrame shape:", flight_df.shape)
    print("\nFinal DataFrame Info:")
    print(flight_df.info())

    print("\nMissing Values After All Processing:")
    print(flight_df.isnull().sum())

if __name__ == "__main__":
    main()


Processing file: 1.docx
Processing file: 10.docx
Processing file: 11.docx
Processing file: 12.docx
Processing file: 13.docx
Processing file: 14.docx
Processing file: 15.docx
Processing file: 16.docx
Processing file: 17.docx
Processing file: 18.docx
Processing file: 19.docx
Processing file: 2.docx
Processing file: 20.docx
Processing file: 21.docx
Processing file: 22.docx
Processing file: 23.docx
Processing file: 24.docx
Processing file: 25.docx
Processing file: 26.docx
Processing file: 27.docx
Processing file: 28.docx
Processing file: 29.docx
Processing file: 3.docx
Processing file: 30.docx
Processing file: 31.docx
Processing file: 32.docx
Processing file: 33.docx
Processing file: 34.docx
Processing file: 35.docx
Processing file: 36.docx
Processing file: 37.docx
Processing file: 39.docx
Processing file: 4.docx
Processing file: 40.docx
Processing file: 41.docx
Processing file: 42.docx
Processing file: 43.docx
Processing file: 44.docx
Processing file: 45.docx
Processing file: 46.docx
Proc

In [3]:
data=pd.read_csv('final_cleaned_flight_data_with_weather.csv')
data

,airline.iataCode,airline.icaoCode,airline_airblue,airline_airsial,airline_british airways,airline_emirates,airline_flyjinnah,airline_klm,airline_oman air,airline_pakistan international airlines,...,departure.scheduledTime,departure.terminal,flight.iataNumber,flight.icaoNumber,flight.number,status,status_encoded,type,departure.delay_minutes,Departure Date
0,sv,sva,False,False,False,False,False,False,False,False,...,2023-07-20 20:50:00,m,sv737,sva737,737,active,0,departure,-35.0,2023-07-20
1,9p,fjl,False,False,False,False,True,False,False,False,...,2023-07-18 15:05:00,Unknown,9p843,fjl843,843,active,0,departure,0.0,2023-07-18
2,9p,fjl,False,False,False,False,True,False,False,False,...,2023-07-23 09:50:00,Unknown,9p841,fjl841,841,active,0,departure,237370.0,2023-07-23
3,pk,pia,False,False,False,False,False,False,False,True,...,2023-07-26 23:30:00,m,pk205,pia205,205,active,0,departure,21.0,2023-07-26
4,er,sep,False,False,False,False,False,False,False,False,...,2023-07-20 11:35:00,m,er723,sep723,723,active,0,departure,340.0,2023-07-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51567,9p,fjl,False,False,False,False,True,False,False,False,...,2023-11-16 09:50:00,Unknown,9p841,fjl841,841,active,0,departure,13.0,2023-11-16
51568,sv,sva,False,False,False,False,False,False,False,False,...,2023-11-16 11:40:00,m,sv735,sva735,735,active,0,departure,8.0,2023-11-16
51569,pk,pia,False,False,False,False,False,False,False,True,...,2023-11-27 10:50:00,m,pk203,pia203,203,active,0,departure,225.0,2023-11-27
51570,pk,pia,False,False,False,False,False,False,False,True,...,2023-11-19 02:00:00,m,pk898,pia898,898,active,0,departure,540.0,2023-11-19


In [5]:
data.columns

Index(['airline.iataCode', 'airline.icaoCode', 'airline_airblue',
       'airline_airsial', 'airline_british airways', 'airline_emirates',
       'airline_flyjinnah', 'airline_klm', 'airline_oman air',
       'airline_pakistan international airlines', 'airline_qatar airways',
       'airline_serene air', 'arrival.estimatedTime', 'arrival.iataCode',
       'arrival.icaoCode', 'arrival.scheduledTime', 'departure.actualRunway',
       'departure.actualTime', 'departure.day_of_week_Monday',
       'departure.day_of_week_Saturday', 'departure.day_of_week_Sunday',
       'departure.day_of_week_Thursday', 'departure.day_of_week_Tuesday',
       'departure.day_of_week_Wednesday', 'departure.estimatedRunway',
       'departure.estimatedTime', 'departure.hour_of_day',
       'departure.iataCode', 'departure.icaoCode', 'departure.month',
       'departure.scheduledTime', 'departure.terminal', 'flight.iataNumber',
       'flight.icaoNumber', 'flight.number', 'status', 'status_encoded',
       'typ

In [13]:
import os
import re
import json
import pandas as pd
from docx import Document
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import joblib

def extract_json_from_docx(file_path):
    """
    Extracts JSON string from a .docx file.
    Assumes that the JSON content is enclosed within [ ] brackets.
    """
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

    full_text = []
    json_started = False

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue  # Skip empty paragraphs

        # Detect the start of JSON array
        if text.startswith('['):
            json_started = True

        if json_started:
            full_text.append(text)
            # Detect the end of JSON array
            if text.endswith(']'):
                break

    json_str = '\n'.join(full_text)

    # Optional: Clean up the JSON string using regex if necessary
    # For example, remove unwanted characters or fix formatting issues
    json_str = re.sub(r'(?<!\\)"', r'"', json_str)  # Replace unescaped quotes if necessary

    return json_str

def parse_json_to_dataframe(json_str):
    """
    Parses a JSON string to a pandas DataFrame.
    Handles both single JSON array and multiple JSON objects.
    """
    if not json_str:
        return None

    try:
        # Attempt to load the JSON string as a list
        data = json.loads(json_str)
        if isinstance(data, list):
            df = pd.json_normalize(data, sep='.')
            return df
        else:
            # If not a list, wrap it into a list
            df = pd.json_normalize([data], sep='.')
            return df
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return None

def process_docx_files(directory_path):
    """
    Processes all .docx files in the specified directory.
    Returns a combined pandas DataFrame.
    """
    all_dfs = []
    
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.docx'):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")
            json_str = extract_json_from_docx(file_path)
            df = parse_json_to_dataframe(json_str)
            if df is not None:
                all_dfs.append(df)
            else:
                print(f"Failed to parse JSON in file: {filename}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        return combined_df
    else:
        print("No DataFrames to concatenate.")
        return pd.DataFrame()

def encode_airline_names(df, top_n=10):
    """
    Encodes 'airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    if 'airline.name' in df.columns:
        top_airlines = df['airline.name'].value_counts().nlargest(top_n).index
        df['airline.name'] = df['airline.name'].apply(lambda x: x if x in top_airlines else 'Other')
        df = pd.get_dummies(df, columns=['airline.name'], prefix='airline', drop_first=True)
        print(f"One-Hot Encoded 'airline.name' with top {top_n} categories.")
    return df

def encode_codeshare_airline_names(df, top_n=10):
    """
    Encodes 'codeshared.airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    # Adjust the column name based on actual DataFrame columns
    possible_column_names = ['codeshared.airline.name', 'codeshare.airline.name', 'codeshare_airline.name']
    column_found = False
    for col in possible_column_names:
        if col in df.columns:
            top_airlines = df[col].value_counts().nlargest(top_n).index
            df[col] = df[col].apply(lambda x: x if x in top_airlines else 'Other')
            df = pd.get_dummies(df, columns=[col], prefix='codeshare_airline', drop_first=True)
            print(f"One-Hot Encoded '{col}' with top {top_n} categories.")
            column_found = True
            break
    if not column_found:
        print("No 'codeshared.airline.name' column found to encode.")
    return df

def encode_categorical_variables(df):
    """
    Encodes categorical variables into numerical formats.
    """
    # Initialize LabelEncoder
    le = LabelEncoder()

    # Encode 'status' column (binary or multi-class classification)
    if 'status' in df.columns:
        df['status_encoded'] = le.fit_transform(df['status'])
        print("Encoded 'status' column.")

    # One-Hot Encode 'departure.day_of_week' if exists
    if 'departure.day_of_week' in df.columns:
        df = pd.get_dummies(df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # One-Hot Encode 'airline.name' with top categories
    df = encode_airline_names(df, top_n=10)

    return df

def encode_all_categorical_variables(df, top_n=10):
    """
    Encodes all relevant categorical variables into numerical formats.
    """
    df = encode_categorical_variables(df)
    df = encode_codeshare_airline_names(df, top_n=top_n)
    return df

def drop_high_missing_columns(df, threshold=50):
    """
    Drops columns from the DataFrame where the percentage of missing values exceeds the threshold.
    """
    missing_percentage = df.isnull().mean() * 100
    cols_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped columns with >{threshold}% missing values: {cols_to_drop}")
    return df

def impute_datetime_columns(df, datetime_cols):
    """
    Converts specified columns to datetime and imputes missing values based on a logical hierarchy.
    """
    for col in datetime_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
            print(f"Converted '{col}' to datetime.")

    # Impute 'departure.estimatedTime' and 'arrival.estimatedTime' with median
    for col in ['departure.estimatedTime', 'arrival.estimatedTime']:
        if col in df.columns and df[col].isnull().sum() > 0:
            median_time = df[col].median()
            df[col].fillna(median_time, inplace=True)
            print(f"Imputed missing '{col}' with median time {median_time}.")

    # Now, impute 'departure.actualTime' with 'departure.estimatedTime'
    if 'departure.actualTime' in df.columns and 'departure.estimatedTime' in df.columns:
        df['departure.actualTime'].fillna(df['departure.estimatedTime'], inplace=True)
        print("Imputed missing 'departure.actualTime' with 'departure.estimatedTime'.")

    # Similarly, impute 'arrival.actualTime' with 'arrival.estimatedTime'
    if 'arrival.actualTime' in df.columns and 'arrival.estimatedTime' in df.columns:
        df['arrival.actualTime'].fillna(df['arrival.estimatedTime'], inplace=True)
        print("Imputed missing 'arrival.actualTime' with 'arrival.estimatedTime'.")

    return df

def impute_remaining_columns(df):
    """
    Imputes remaining numerical columns with median and categorical columns with 'Unknown'.
    """
    # Identify numerical and categorical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    # Impute numerical columns with median
    for col in numerical_cols:
        if df[col].isnull().sum() > 0:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"Imputed missing values in numerical column '{col}' with median value {median_value}.")

    # Impute categorical columns with 'Unknown'
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna('Unknown', inplace=True)
            print(f"Imputed missing values in categorical column '{col}' with 'Unknown'.")

    return df

def remove_low_variance_features(df, threshold=0.0):
    """
    Removes numerical features with variance below the specified threshold.
    Non-numerical columns are retained without modification.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - threshold (float): The variance threshold.
    
    Returns:
    - pd.DataFrame: The DataFrame with low variance numerical features removed.
    """
    # Select numerical columns (int64, float64, uint8)
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()

    if not numeric_cols:
        print("No numerical columns to apply VarianceThreshold.")
        return df

    # Initialize VarianceThreshold
    selector = VarianceThreshold(threshold=threshold)

    # Fit the selector on numerical data
    try:
        selector.fit(df[numeric_cols])
    except ValueError as e:
        print(f"VarianceThreshold failed: {e}")
        return df

    # Get the columns to keep
    features_to_keep = [col for col, keep in zip(numeric_cols, selector.get_support()) if keep]

    # Retain selected numerical features and all non-numerical features
    non_numeric_cols = df.columns.difference(numeric_cols)
    df_filtered = pd.concat([df[non_numeric_cols], df[features_to_keep]], axis=1)

    print(f"Removed low variance features. Remaining features: {len(df_filtered.columns)}")
    
    return df_filtered

def load_and_preprocess_weather_data(weather_directory, num_files=13):
    """
    Loads, reshapes, and concatenates weather data from multiple Excel files.
    
    Parameters:
    - weather_directory (str): Path to the directory containing weather Excel files.
    - num_files (int): Number of Excel files to load (e.g., first 13 files).
    
    Returns:
    - pd.DataFrame: Combined and reshaped weather data.
    """
    weather_dfs = []
    start_month = 7  # Assuming file 1.xlsx is July
    start_year = 2023  # Starting year

    for i in range(1, num_files + 1):
        file_name = f"{i}.xlsx"
        file_path = os.path.join(weather_directory, file_name)
        if os.path.exists(file_path):
            try:
                df_weather = pd.read_excel(file_path, header=None)
                print(f"Loaded weather data from {file_name}")
                
                # Determine month and year
                month = (start_month + i - 1) % 12 + 1
                year = start_year + (start_month + i - 1) // 12
                
                # Reshape weather data with correct year and month
                reshaped_weather = reshape_weather_data(df_weather, year=year, month=month)
                
                if not reshaped_weather.empty:
                    weather_dfs.append(reshaped_weather)
                else:
                    print(f"Reshaped weather data from {file_name} is empty. Skipping.")
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
        else:
            print(f"Weather file {file_name} does not exist in {weather_directory}. Skipping.")
    
    if weather_dfs:
        combined_weather_df = pd.concat(weather_dfs, ignore_index=True)
        print(f"Combined weather DataFrame shape: {combined_weather_df.shape}")
        return combined_weather_df
    else:
        print("No weather data files loaded or reshaped successfully.")
        return pd.DataFrame()

def reshape_weather_data(weather_df, year=2023, month=7):
    """
    Reshapes the weather data from wide to long format, handling multi-row metrics.

    Assumes that:
    - The first row contains 'Time' and day identifiers (e.g., 'Jul 1', '2', '3', ..., '14')
    - Subsequent rows contain metrics and their submetrics (e.g., 'Temperature (°F)', 'Max', 'Avg', 'Min')

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data without headers.
    - year (int): Year for the dates.
    - month (int): Month number for the dates.

    Returns:
    - pd.DataFrame: Reshaped weather data with one row per day and separate columns for each metric.
    """
    # Initialize
    data = []
    current_metric = None
    submetrics = ['Max', 'Avg', 'Min', 'Total']  # Include 'Total' for Precipitation

    # Extract day identifiers from the first row
    days = weather_df.iloc[0, 1:].tolist()  # Skip first column 'Time'

    # Convert day numbers to proper day strings
    month_abbr = pd.to_datetime(f'{month}', format='%m').strftime('%b')
    processed_days = []
    for day in days:
        if isinstance(day, str) and re.match(r'^[A-Za-z]+', day):
            processed_days.append(day)
        else:
            processed_days.append(f"{month_abbr} {day}")

    # Iterate through the rest of the rows
    for index, row in weather_df.iloc[1:].iterrows():
        metric = row[0]
        if pd.isnull(metric):
            continue  # Skip empty rows

        if metric not in submetrics:
            # This row defines a new main metric
            current_metric = metric
            continue

        # This row defines a submetric under the current main metric
        submetric = metric
        if current_metric is None:
            print(f"Submetric '{submetric}' found without a main metric at row {index}. Skipping.")
            continue

        # Iterate through day columns
        for day_idx, value in enumerate(row[1:]):
            if day_idx >= len(processed_days):
                continue  # Skip if day index exceeds available days

            date_str = processed_days[day_idx]
            try:
                date = pd.to_datetime(f"{date_str} {year}", format='%b %d %Y', errors='coerce')
            except Exception as e:
                print(f"Error parsing date '{date_str} {year}': {e}")
                date = pd.NaT

            if pd.isnull(date):
                continue  # Skip invalid dates

            # Define full metric name
            if submetric == 'Total':
                full_metric = 'Precipitation (in)'
            else:
                full_metric = f"{current_metric} ({submetric})"

            # Append to data
            data.append({'Date': date, 'Metric': full_metric, 'Value': value})

    # Create long-format DataFrame
    weather_long_df = pd.DataFrame(data)

    # Pivot to wide format
    if weather_long_df.empty:
        print("No valid weather data to reshape.")
        return pd.DataFrame()

    weather_pivot = weather_long_df.pivot_table(
        index='Date',
        columns='Metric',
        values='Value'
    ).reset_index()

    # Convert metric columns to numeric types
    for col in weather_pivot.columns:
        if col != 'Date':
            weather_pivot[col] = pd.to_numeric(weather_pivot[col], errors='coerce')

    print(f"Reshaped weather DataFrame shape: {weather_pivot.shape}")
    return weather_pivot

def preprocess_weather_data(weather_df, year=2023, month=7):
    """
    Preprocesses the weather data DataFrame.

    - Reshapes the data from wide to long format.
    - Parses 'Date' column to datetime.
    - Handles missing values if any.

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data without headers.
    - year (int): Year for the dates.
    - month (int): Month number for the dates.

    Returns:
    - pd.DataFrame: Preprocessed weather data.
    """
    # Reshape the weather data
    weather_pivot = reshape_weather_data(weather_df, year=year, month=month)

    # Verify the reshaped data
    if weather_pivot.empty:
        print("Reshaping resulted in an empty DataFrame.")
        return weather_pivot

    return weather_pivot

def merge_flight_weather(flight_df, weather_df):
    """
    Merges flight data with weather data based on departure date.
    
    Parameters:
    - flight_df (pd.DataFrame): Cleaned flight data with 'departure.scheduledTime'.
    - weather_df (pd.DataFrame): Cleaned and reshaped weather data with 'Date'.
    
    Returns:
    - pd.DataFrame: Merged DataFrame.
    """
    # Ensure 'departure.scheduledTime' is datetime
    if not pd.api.types.is_datetime64_any_dtype(flight_df['departure.scheduledTime']):
        flight_df['departure.scheduledTime'] = pd.to_datetime(flight_df['departure.scheduledTime'], errors='coerce')
        print("Converted 'departure.scheduledTime' to datetime.")
    
    # Create a 'Departure Date' column (date only, no time)
    flight_df['Departure Date'] = flight_df['departure.scheduledTime'].dt.date
    weather_df['Date'] = weather_df['Date'].dt.date  # Ensure 'Date' is date only

    # Merge on 'Departure Date' and 'Date'
    merged_df = pd.merge(
        flight_df,
        weather_df,
        how='left',
        left_on='Departure Date',
        right_on='Date',
        suffixes=('', '_weather')
    )

    # Drop the redundant 'Date' column from weather data
    merged_df.drop(columns=['Date'], inplace=True)

    print(f"Merged flight data with weather data. Merged DataFrame shape: {merged_df.shape}")
    return merged_df

def encode_and_save_label_encoder(df, column, encoder_filename):
    """
    Encodes a categorical column using LabelEncoder and saves the encoder.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the column.
    - column (str): The name of the column to encode.
    - encoder_filename (str): The filename to save the encoder.
    
    Returns:
    - pd.DataFrame: The DataFrame with the encoded column.
    """
    if column in df.columns:
        le = LabelEncoder()
        df[f'{column}_encoded'] = le.fit_transform(df[column])
        joblib.dump(le, encoder_filename)
        print(f"Encoded '{column}' column and saved LabelEncoder as '{encoder_filename}'.")
    return df

def main():
    # Specify the directories containing your data
    flight_data_directory = 'ML-Proj-Dataset/Train/'    # Replace with your actual flight data path
    weather_data_directory = 'ML-Proj-Dataset/Weather/'         # Replace with your actual weather data path

    # Check if the directories exist
    if not os.path.isdir(flight_data_directory):
        print(f"The directory '{flight_data_directory}' does not exist. Please check the path.")
        return
    if not os.path.isdir(weather_data_directory):
        print(f"The directory '{weather_data_directory}' does not exist. Please check the path.")
        return

    # Process the .docx flight data files and get the combined DataFrame
    flight_df = process_docx_files(flight_data_directory)

    if flight_df.empty:
        print("No flight data to process.")
        return

    print(f"\nInitial Flight DataFrame shape: {flight_df.shape}")

    # Define datetime columns in flight data
    flight_datetime_cols = [
        'departure.scheduledTime',
        'departure.estimatedTime',
        'departure.actualTime',
        'departure.estimatedRunway',
        'departure.actualRunway',
        'arrival.scheduledTime',
        'arrival.estimatedTime',
        'arrival.actualTime',
        'arrival.estimatedRunway',
        'arrival.actualRunway'
    ]

    # Drop columns with >50% missing values
    flight_df = drop_high_missing_columns(flight_df, threshold=50)

    # Convert and impute datetime columns
    flight_df = impute_datetime_columns(flight_df, flight_datetime_cols)

    # Impute remaining numerical and categorical columns
    flight_df = impute_remaining_columns(flight_df)

    # Encode categorical variables with optimized methods
    flight_df = encode_all_categorical_variables(flight_df, top_n=10)

    # Remove low variance numerical features
    flight_df = remove_low_variance_features(flight_df, threshold=0.01)  # Adjust threshold as needed

    # Calculate departure delay if possible
    if 'departure.actualTime' in flight_df.columns and 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.delay_minutes'] = (flight_df['departure.actualTime'] - flight_df['departure.scheduledTime']).dt.total_seconds() / 60
        print("Calculated 'departure.delay_minutes'.")

    # Extract temporal features if needed
    if 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.day_of_week'] = flight_df['departure.scheduledTime'].dt.day_name()
        flight_df['departure.hour_of_day'] = flight_df['departure.scheduledTime'].dt.hour
        flight_df['departure.month'] = flight_df['departure.scheduledTime'].dt.month
        print("Extracted temporal features.")

        # One-Hot Encode temporal features
        flight_df = pd.get_dummies(flight_df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # Remove low variance numerical features again after adding temporal features
    flight_df = remove_low_variance_features(flight_df, threshold=0.0)  # Remove features with zero variance

    # Impute 'departure.delay_minutes' with median if there are still missing values
    if 'departure.delay_minutes' in flight_df.columns and flight_df['departure.delay_minutes'].isnull().sum() > 0:
        median_delay = flight_df['departure.delay_minutes'].median()
        flight_df['departure.delay_minutes'].fillna(median_delay, inplace=True)
        print(f"Imputed missing 'departure.delay_minutes' with median value {median_delay}.")

    # Load and preprocess weather data
    weather_df = load_and_preprocess_weather_data(weather_data_directory, num_files=13)
    if weather_df.empty:
        print("No weather data available for merging.")
    else:
        # Merge flight data with weather data
        flight_df = merge_flight_weather(flight_df, weather_df)

        # Handle any remaining missing weather data by imputing with median
        weather_metrics = ['Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)', 
                           'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)']
        
        for metric in weather_metrics:
            if metric in flight_df.columns:
                if flight_df[metric].isnull().sum() > 0:
                    median_value = flight_df[metric].median()
                    flight_df[metric].fillna(median_value, inplace=True)
                    print(f"Imputed missing '{metric}' with median value {median_value}.")

    # Save LabelEncoder for 'status'
    if 'status_encoded' in flight_df.columns:
        le_status = LabelEncoder()
        le_status.fit(flight_df['status'])
        joblib.dump(le_status, 'label_encoder_status.pkl')
        print("Saved LabelEncoder for 'status' column as 'label_encoder_status.pkl'.")

    # Save the final cleaned and merged DataFrame
    flight_df.to_csv('final_cleaned_flight_data_with_weather.csv', index=False)
    print("\nFinal cleaned and merged DataFrame saved as 'final_cleaned_flight_data_with_weather.csv'.")

    # Display final DataFrame information
    print("\nFinal DataFrame shape:", flight_df.shape)
    print("\nFinal DataFrame Info:")
    print(flight_df.info())

    print("\nMissing Values After All Processing:")
    print(flight_df.isnull().sum())

if __name__ == "__main__":
    main()


Processing file: 1.docx
Processing file: 10.docx
Processing file: 11.docx
Processing file: 12.docx
Processing file: 13.docx
Processing file: 14.docx
Processing file: 15.docx
Processing file: 16.docx
Processing file: 17.docx
Processing file: 18.docx
Processing file: 19.docx
Processing file: 2.docx
Processing file: 20.docx
Processing file: 21.docx
Processing file: 22.docx
Processing file: 23.docx
Processing file: 24.docx
Processing file: 25.docx
Processing file: 26.docx
Processing file: 27.docx
Processing file: 28.docx
Processing file: 29.docx
Processing file: 3.docx
Processing file: 30.docx
Processing file: 31.docx
Processing file: 32.docx
Processing file: 33.docx
Processing file: 34.docx
Processing file: 35.docx
Processing file: 36.docx
Processing file: 37.docx
Processing file: 39.docx
Processing file: 4.docx
Processing file: 40.docx
Processing file: 41.docx
Processing file: 42.docx
Processing file: 43.docx
Processing file: 44.docx
Processing file: 45.docx
Processing file: 46.docx
Proc

In [17]:
import os
import re
import json
import pandas as pd
from docx import Document
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import joblib

def extract_json_from_docx(file_path):
    """
    Extracts JSON string from a .docx file.
    Assumes that the JSON content is enclosed within [ ] brackets.
    """
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

    full_text = []
    json_started = False

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue  # Skip empty paragraphs

        # Detect the start of JSON array
        if text.startswith('['):
            json_started = True

        if json_started:
            full_text.append(text)
            # Detect the end of JSON array
            if text.endswith(']'):
                break

    json_str = '\n'.join(full_text)

    # Optional: Clean up the JSON string using regex if necessary
    # For example, remove unwanted characters or fix formatting issues
    json_str = re.sub(r'(?<!\\)"', r'"', json_str)  # Replace unescaped quotes if necessary

    return json_str

def parse_json_to_dataframe(json_str):
    """
    Parses a JSON string to a pandas DataFrame.
    Handles both single JSON array and multiple JSON objects.
    """
    if not json_str:
        return None

    try:
        # Attempt to load the JSON string as a list
        data = json.loads(json_str)
        if isinstance(data, list):
            df = pd.json_normalize(data, sep='.')
            return df
        else:
            # If not a list, wrap it into a list
            df = pd.json_normalize([data], sep='.')
            return df
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return None

def process_docx_files(directory_path):
    """
    Processes all .docx files in the specified directory.
    Returns a combined pandas DataFrame.
    """
    all_dfs = []
    
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.docx'):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")
            json_str = extract_json_from_docx(file_path)
            df = parse_json_to_dataframe(json_str)
            if df is not None:
                all_dfs.append(df)
            else:
                print(f"Failed to parse JSON in file: {filename}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        return combined_df
    else:
        print("No DataFrames to concatenate.")
        return pd.DataFrame()

def encode_airline_names(df, top_n=10):
    """
    Encodes 'airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    if 'airline.name' in df.columns:
        top_airlines = df['airline.name'].value_counts().nlargest(top_n).index
        df['airline.name'] = df['airline.name'].apply(lambda x: x if x in top_airlines else 'Other')
        df = pd.get_dummies(df, columns=['airline.name'], prefix='airline', drop_first=True)
        print(f"One-Hot Encoded 'airline.name' with top {top_n} categories.")
    return df

def encode_codeshare_airline_names(df, top_n=10):
    """
    Encodes 'codeshared.airline.name' by one-hot encoding the top N frequent airlines.
    Groups the rest as 'Other'.
    """
    # Adjust the column name based on actual DataFrame columns
    possible_column_names = ['codeshared.airline.name', 'codeshare.airline.name', 'codeshare_airline.name']
    column_found = False
    for col in possible_column_names:
        if col in df.columns:
            top_airlines = df[col].value_counts().nlargest(top_n).index
            df[col] = df[col].apply(lambda x: x if x in top_airlines else 'Other')
            df = pd.get_dummies(df, columns=[col], prefix='codeshare_airline', drop_first=True)
            print(f"One-Hot Encoded '{col}' with top {top_n} categories.")
            column_found = True
            break
    if not column_found:
        print("No 'codeshared.airline.name' column found to encode.")
    return df

def encode_categorical_variables(df):
    """
    Encodes categorical variables into numerical formats.
    """
    # Initialize LabelEncoder
    le = LabelEncoder()

    # Encode 'status' column (binary or multi-class classification)
    if 'status' in df.columns:
        df['status_encoded'] = le.fit_transform(df['status'])
        print("Encoded 'status' column.")

    # One-Hot Encode 'departure.day_of_week' if exists
    if 'departure.day_of_week' in df.columns:
        df = pd.get_dummies(df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # One-Hot Encode 'airline.name' with top categories
    df = encode_airline_names(df, top_n=10)

    return df

def encode_all_categorical_variables(df, top_n=10):
    """
    Encodes all relevant categorical variables into numerical formats.
    """
    df = encode_categorical_variables(df)
    df = encode_codeshare_airline_names(df, top_n=top_n)
    return df

def drop_high_missing_columns(df, threshold=50):
    """
    Drops columns from the DataFrame where the percentage of missing values exceeds the threshold.
    """
    missing_percentage = df.isnull().mean() * 100
    cols_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped columns with >{threshold}% missing values: {cols_to_drop}")
    return df

def impute_datetime_columns(df, datetime_cols):
    """
    Converts specified columns to datetime and imputes missing values based on a logical hierarchy.
    """
    for col in datetime_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
            print(f"Converted '{col}' to datetime.")

    # Impute 'departure.estimatedTime' and 'arrival.estimatedTime' with median
    for col in ['departure.estimatedTime', 'arrival.estimatedTime']:
        if col in df.columns and df[col].isnull().sum() > 0:
            median_time = df[col].median()
            df[col].fillna(median_time, inplace=True)
            print(f"Imputed missing '{col}' with median time {median_time}.")

    # Now, impute 'departure.actualTime' with 'departure.estimatedTime'
    if 'departure.actualTime' in df.columns and 'departure.estimatedTime' in df.columns:
        df['departure.actualTime'].fillna(df['departure.estimatedTime'], inplace=True)
        print("Imputed missing 'departure.actualTime' with 'departure.estimatedTime'.")

    # Similarly, impute 'arrival.actualTime' with 'arrival.estimatedTime'
    if 'arrival.actualTime' in df.columns and 'arrival.estimatedTime' in df.columns:
        df['arrival.actualTime'].fillna(df['arrival.estimatedTime'], inplace=True)
        print("Imputed missing 'arrival.actualTime' with 'arrival.estimatedTime'.")

    return df

def impute_remaining_columns(df):
    """
    Imputes remaining numerical columns with median and categorical columns with 'Unknown'.
    """
    # Identify numerical and categorical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    # Impute numerical columns with median
    for col in numerical_cols:
        if df[col].isnull().sum() > 0:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"Imputed missing values in numerical column '{col}' with median value {median_value}.")

    # Impute categorical columns with 'Unknown'
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna('Unknown', inplace=True)
            print(f"Imputed missing values in categorical column '{col}' with 'Unknown'.")

    return df

def remove_low_variance_features(df, threshold=0.0):
    """
    Removes numerical features with variance below the specified threshold.
    Non-numerical columns are retained without modification.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - threshold (float): The variance threshold.
    
    Returns:
    - pd.DataFrame: The DataFrame with low variance numerical features removed.
    """
    # Select numerical columns (int64, float64, uint8)
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'uint8']).columns.tolist()

    if not numeric_cols:
        print("No numerical columns to apply VarianceThreshold.")
        return df

    # Initialize VarianceThreshold
    selector = VarianceThreshold(threshold=threshold)

    # Fit the selector on numerical data
    try:
        selector.fit(df[numeric_cols])
    except ValueError as e:
        print(f"VarianceThreshold failed: {e}")
        return df

    # Get the columns to keep
    features_to_keep = [col for col, keep in zip(numeric_cols, selector.get_support()) if keep]

    # Retain selected numerical features and all non-numerical features
    non_numeric_cols = df.columns.difference(numeric_cols)
    df_filtered = pd.concat([df[non_numeric_cols], df[features_to_keep]], axis=1)

    print(f"Removed low variance features. Remaining features: {len(df_filtered.columns)}")
    
    return df_filtered

def load_and_preprocess_weather_data(weather_directory, num_files=13):
    """
    Loads, reshapes, and concatenates weather data from multiple Excel files.
    
    Parameters:
    - weather_directory (str): Path to the directory containing weather Excel files.
    - num_files (int): Number of Excel files to load (e.g., first 13 files).
    
    Returns:
    - pd.DataFrame: Combined and reshaped weather data.
    """
    weather_dfs = []
    start_month = 7  # Assuming file 1.xlsx is July
    start_year = 2023  # Starting year

    for i in range(1, num_files + 1):
        file_name = f"{i}.xlsx"
        file_path = os.path.join(weather_directory, file_name)
        if os.path.exists(file_path):
            try:
                # Read the Excel file without headers
                df_weather = pd.read_excel(file_path, header=None)
                print(f"Loaded weather data from {file_name}")
                
                # Determine month and year
                month = (start_month + i - 1) % 12 + 1
                year = start_year + (start_month + i - 1) // 12
                
                # Reshape weather data with correct year and month
                reshaped_weather = reshape_weather_data(df_weather, year=year, month=month)
                
                if not reshaped_weather.empty:
                    weather_dfs.append(reshaped_weather)
                else:
                    print(f"Reshaped weather data from {file_name} is empty. Skipping.")
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
        else:
            print(f"Weather file {file_name} does not exist in {weather_directory}. Skipping.")
    
    if weather_dfs:
        combined_weather_df = pd.concat(weather_dfs, ignore_index=True)
        print(f"Combined weather DataFrame shape: {combined_weather_df.shape}")
        return combined_weather_df
    else:
        print("No weather data files loaded or reshaped successfully.")
        return pd.DataFrame()

def reshape_weather_data(weather_df, year=2023, month=7):
    """
    Reshapes the weather data from wide to long format, handling multi-row metrics.

    Assumes that:
    - The first row contains 'Time' and day identifiers (e.g., 'Jul 1', '2', '3', ..., '14')
    - Subsequent rows contain metrics and their submetrics (e.g., 'Temperature (°F)', 'Max', 'Avg', 'Min')
    - Submetrics are listed in a single cell separated by spaces (e.g., 'Max Avg Min')

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data without headers.
    - year (int): Year for the dates.
    - month (int): Month number for the dates.

    Returns:
    - pd.DataFrame: Reshaped weather data with one row per day and separate columns for each metric.
    """
    data = []
    current_metric = None
    submetrics = []

    # Extract day identifiers from the second row
    days_row = weather_df.iloc[1, 1:].tolist()  # Skip first column 'Time'
    month_abbr = pd.to_datetime(f'{month}', format='%m').strftime('%b')
    processed_days = []
    for day in days_row:
        if isinstance(day, str) and re.match(r'^[A-Za-z]+', day):
            processed_days.append(day)
        else:
            processed_days.append(f"{month_abbr} {day}")

    # Iterate over the data starting from the third row
    row_idx = 2
    total_rows = weather_df.shape[0]
    while row_idx < total_rows:
        metric_name = weather_df.iloc[row_idx, 0]
        if pd.isnull(metric_name):
            row_idx += 1
            continue  # Skip empty rows
        current_metric = metric_name
        row_idx += 1
        if row_idx >= total_rows:
            break  # Prevent index out of range

        submetric_row = weather_df.iloc[row_idx, :].tolist()
        submetric_names = []
        submetric_values = []
        
        # Assuming the first cell contains all submetric names separated by space
        submetrics_cell = submetric_row[0]
        if isinstance(submetrics_cell, str):
            submetric_names = submetrics_cell.split()
        else:
            print(f"Unexpected submetric format at row {row_idx}: {submetrics_cell}")
            row_idx += 1
            continue

        # The rest of the cells contain values
        values = submetric_row[1:]
        # Ensure that the number of values matches the number of submetrics times the number of days
        expected_values = len(submetric_names) * len(processed_days)
        if len(values) < expected_values:
            print(f"Not enough values for metric '{current_metric}'. Expected {expected_values}, got {len(values)}.")
            row_idx += 1
            continue

        # Iterate through each day and submetric
        for day_idx, day in enumerate(processed_days):
            for sub_idx, submetric in enumerate(submetric_names):
                value_idx = day_idx * len(submetric_names) + sub_idx
                value = values[value_idx]
                try:
                    date = pd.to_datetime(f"{day} {year}", format='%b %d %Y', errors='coerce')
                    if pd.isnull(date):
                        print(f"Invalid date: {day} {year}")
                        continue
                except Exception as e:
                    print(f"Error parsing date '{day} {year}': {e}")
                    continue

                # Define full metric name
                if submetric.lower() == 'total':
                    full_metric = 'Precipitation (in)'
                else:
                    full_metric = f"{current_metric} ({submetric})"

                # Append to data
                data.append({'Date': date, 'Metric': full_metric, 'Value': value})

        row_idx += 1  # Move to the next metric

    # Create long-format DataFrame
    weather_long_df = pd.DataFrame(data)

    # Pivot to wide format
    if weather_long_df.empty:
        print("No valid weather data to reshape.")
        return pd.DataFrame()

    weather_pivot = weather_long_df.pivot_table(
        index='Date',
        columns='Metric',
        values='Value'
    ).reset_index()

    # Convert metric columns to numeric types
    for col in weather_pivot.columns:
        if col != 'Date':
            weather_pivot[col] = pd.to_numeric(weather_pivot[col], errors='coerce')

    print(f"Reshaped weather DataFrame shape: {weather_pivot.shape}")
    return weather_pivot

def preprocess_weather_data(weather_df, year=2023, month=7):
    """
    Preprocesses the weather data DataFrame.

    - Reshapes the data from wide to long format.
    - Parses 'Date' column to datetime.
    - Handles missing values if any.

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data without headers.
    - year (int): Year for the dates.
    - month (int): Month number for the dates.

    Returns:
    - pd.DataFrame: Preprocessed weather data.
    """
    # Reshape the weather data
    weather_pivot = reshape_weather_data(weather_df, year=year, month=month)

    # Verify the reshaped data
    if weather_pivot.empty:
        print("Reshaping resulted in an empty DataFrame.")
        return weather_pivot

    return weather_pivot

def merge_flight_weather(flight_df, weather_df):
    """
    Merges flight data with weather data based on departure date.
    
    Parameters:
    - flight_df (pd.DataFrame): Cleaned flight data with 'departure.scheduledTime'.
    - weather_df (pd.DataFrame): Cleaned and reshaped weather data with 'Date'.
    
    Returns:
    - pd.DataFrame: Merged DataFrame.
    """
    # Ensure 'departure.scheduledTime' is datetime
    if not pd.api.types.is_datetime64_any_dtype(flight_df['departure.scheduledTime']):
        flight_df['departure.scheduledTime'] = pd.to_datetime(flight_df['departure.scheduledTime'], errors='coerce')
        print("Converted 'departure.scheduledTime' to datetime.")
    
    # Create a 'Departure Date' column (date only, no time)
    flight_df['Departure Date'] = flight_df['departure.scheduledTime'].dt.date
    weather_df['Date'] = weather_df['Date'].dt.date  # Ensure 'Date' is date only

    # Merge on 'Departure Date' and 'Date'
    merged_df = pd.merge(
        flight_df,
        weather_df,
        how='left',
        left_on='Departure Date',
        right_on='Date',
        suffixes=('', '_weather')
    )

    # Drop the redundant 'Date' column from weather data
    merged_df.drop(columns=['Date'], inplace=True)

    print(f"Merged flight data with weather data. Merged DataFrame shape: {merged_df.shape}")
    return merged_df

def encode_and_save_label_encoder(df, column, encoder_filename):
    """
    Encodes a categorical column using LabelEncoder and saves the encoder.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the column.
    - column (str): The name of the column to encode.
    - encoder_filename (str): The filename to save the encoder.
    
    Returns:
    - pd.DataFrame: The DataFrame with the encoded column.
    """
    if column in df.columns:
        le = LabelEncoder()
        df[f'{column}_encoded'] = le.fit_transform(df[column])
        joblib.dump(le, encoder_filename)
        print(f"Encoded '{column}' column and saved LabelEncoder as '{encoder_filename}'.")
    return df

def main():
    # Specify the directories containing your data
    flight_data_directory = 'ML-Proj-Dataset/Train/'    # Replace with your actual flight data path
    weather_data_directory = 'ML-Proj-Dataset/Weather/'         # Replace with your actual weather data path

    # Check if the directories exist
    if not os.path.isdir(flight_data_directory):
        print(f"The directory '{flight_data_directory}' does not exist. Please check the path.")
        return
    if not os.path.isdir(weather_data_directory):
        print(f"The directory '{weather_data_directory}' does not exist. Please check the path.")
        return

    # Process the .docx flight data files and get the combined DataFrame
    flight_df = process_docx_files(flight_data_directory)

    if flight_df.empty:
        print("No flight data to process.")
        return

    print(f"\nInitial Flight DataFrame shape: {flight_df.shape}")

    # Define datetime columns in flight data
    flight_datetime_cols = [
        'departure.scheduledTime',
        'departure.estimatedTime',
        'departure.actualTime',
        'departure.estimatedRunway',
        'departure.actualRunway',
        'arrival.scheduledTime',
        'arrival.estimatedTime',
        'arrival.actualTime',
        'arrival.estimatedRunway',
        'arrival.actualRunway'
    ]

    # Drop columns with >50% missing values
    flight_df = drop_high_missing_columns(flight_df, threshold=50)

    # Convert and impute datetime columns
    flight_df = impute_datetime_columns(flight_df, flight_datetime_cols)

    # Impute remaining numerical and categorical columns
    flight_df = impute_remaining_columns(flight_df)

    # Encode categorical variables with optimized methods
    flight_df = encode_all_categorical_variables(flight_df, top_n=10)

    # Remove low variance numerical features
    flight_df = remove_low_variance_features(flight_df, threshold=0.01)  # Adjust threshold as needed

    # Calculate departure delay if possible
    if 'departure.actualTime' in flight_df.columns and 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.delay_minutes'] = (flight_df['departure.actualTime'] - flight_df['departure.scheduledTime']).dt.total_seconds() / 60
        print("Calculated 'departure.delay_minutes'.")

    # Extract temporal features if needed
    if 'departure.scheduledTime' in flight_df.columns:
        flight_df['departure.day_of_week'] = flight_df['departure.scheduledTime'].dt.day_name()
        flight_df['departure.hour_of_day'] = flight_df['departure.scheduledTime'].dt.hour
        flight_df['departure.month'] = flight_df['departure.scheduledTime'].dt.month
        print("Extracted temporal features.")

        # One-Hot Encode temporal features
        flight_df = pd.get_dummies(flight_df, columns=['departure.day_of_week'], drop_first=True)
        print("One-Hot Encoded 'departure.day_of_week'.")

    # Remove low variance numerical features again after adding temporal features
    flight_df = remove_low_variance_features(flight_df, threshold=0.0)  # Remove features with zero variance

    # Impute 'departure.delay_minutes' with median if there are still missing values
    if 'departure.delay_minutes' in flight_df.columns and flight_df['departure.delay_minutes'].isnull().sum() > 0:
        median_delay = flight_df['departure.delay_minutes'].median()
        flight_df['departure.delay_minutes'].fillna(median_delay, inplace=True)
        print(f"Imputed missing 'departure.delay_minutes' with median value {median_delay}.")

    # Load and preprocess weather data
    weather_df = load_and_preprocess_weather_data(weather_data_directory, num_files=13)
    if weather_df.empty:
        print("No weather data available for merging.")
    else:
        # Merge flight data with weather data
        flight_df = merge_flight_weather(flight_df, weather_df)

        # Handle any remaining missing weather data by imputing with median
        weather_metrics = ['Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)', 
                           'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)']
        
        for metric in weather_metrics:
            if metric in flight_df.columns:
                if flight_df[metric].isnull().sum() > 0:
                    median_value = flight_df[metric].median()
                    flight_df[metric].fillna(median_value, inplace=True)
                    print(f"Imputed missing '{metric}' with median value {median_value}.")

    # Save LabelEncoder for 'status'
    if 'status_encoded' in flight_df.columns:
        le_status = LabelEncoder()
        le_status.fit(flight_df['status'])
        joblib.dump(le_status, 'label_encoder_status.pkl')
        print("Saved LabelEncoder for 'status' column as 'label_encoder_status.pkl'.")

    # Save the final cleaned and merged DataFrame
    flight_df.to_csv('final_cleaned_flight_data_with_weather.csv', index=False)
    print("\nFinal cleaned and merged DataFrame saved as 'final_cleaned_flight_data_with_weather.csv'.")

    # Display final DataFrame information
    print("\nFinal DataFrame shape:", flight_df.shape)
    print("\nFinal DataFrame Info:")
    print(flight_df.info())

    print("\nMissing Values After All Processing:")
    print(flight_df.isnull().sum())

if __name__ == "__main__":
    main()


Processing file: 1.docx
Processing file: 10.docx
Processing file: 11.docx
Processing file: 12.docx
Processing file: 13.docx
Processing file: 14.docx
Processing file: 15.docx
Processing file: 16.docx
Processing file: 17.docx
Processing file: 18.docx
Processing file: 19.docx
Processing file: 2.docx
Processing file: 20.docx
Processing file: 21.docx
Processing file: 22.docx
Processing file: 23.docx
Processing file: 24.docx
Processing file: 25.docx
Processing file: 26.docx
Processing file: 27.docx
Processing file: 28.docx
Processing file: 29.docx
Processing file: 3.docx
Processing file: 30.docx
Processing file: 31.docx
Processing file: 32.docx
Processing file: 33.docx
Processing file: 34.docx
Processing file: 35.docx
Processing file: 36.docx
Processing file: 37.docx
Processing file: 39.docx
Processing file: 4.docx
Processing file: 40.docx
Processing file: 41.docx
Processing file: 42.docx
Processing file: 43.docx
Processing file: 44.docx
Processing file: 45.docx
Processing file: 46.docx
Proc

In [21]:
import os
import re
import pandas as pd

def reshape_weather_data(weather_df, year=2023, month=7):
    """
    Reshapes the weather data from wide to long format, handling multi-row metrics.

    Assumes that:
    - The first row contains 'Time' and day identifiers (e.g., 'Jul 1', '2', '3', ..., '14')
    - Each subsequent metric is followed by a single row containing submetrics and their values

    Parameters:
    - weather_df (pd.DataFrame): Raw weather data without headers.
    - year (int): Year for the dates.
    - month (int): Month number for the dates.

    Returns:
    - pd.DataFrame: Reshaped weather data with one row per day and separate columns for each metric.
    """
    data = []
    current_metric = None
    month_abbr = pd.to_datetime(f'{month}', format='%m').strftime('%b')
    
    # Extract day identifiers from the first row
    days_row = weather_df.iloc[0, 1:].tolist()  # Skip first cell 'Time'
    processed_days = []
    for day in days_row:
        if isinstance(day, str) and re.match(r'^[A-Za-z]+', day):
            processed_days.append(day)
        else:
            processed_days.append(f"{month_abbr} {day}")
    
    # Iterate through the rest of the rows in pairs (metric and submetrics)
    for idx in range(1, weather_df.shape[0], 2):
        metric_row = weather_df.iloc[idx, :].tolist()
        if not metric_row or pd.isnull(metric_row[0]):
            continue  # Skip empty rows
        
        current_metric = metric_row[0]
        if current_metric.lower() == 'precipitation (in)':
            submetrics = ['Total']
        else:
            submetrics = ['Max', 'Avg', 'Min']
        
        # Next row contains submetrics and their values
        if idx + 1 >= weather_df.shape[0]:
            print(f"Missing submetrics for metric '{current_metric}' at row {idx}. Skipping.")
            continue
        
        submetric_row = weather_df.iloc[idx + 1, :].tolist()
        if not submetric_row or pd.isnull(submetric_row[0]):
            print(f"Missing submetrics data for metric '{current_metric}' at row {idx + 1}. Skipping.")
            continue
        
        submetrics_in_row = submetric_row[0].split()  # Split submetrics by space
        if len(submetrics_in_row) != len(submetrics):
            print(f"Mismatch in submetrics for metric '{current_metric}'. Expected {len(submetrics)}, got {len(submetrics_in_row)}. Skipping.")
            continue
        
        # Extract values for each submetric
        values = submetric_row[1:]
        expected_values = len(submetrics) * len(processed_days)
        if len(values) < expected_values:
            print(f"Insufficient values for metric '{current_metric}'. Expected {expected_values}, got {len(values)}. Skipping.")
            continue
        
        for day_idx, day in enumerate(processed_days):
            for sub_idx, submetric in enumerate(submetrics):
                value_idx = day_idx * len(submetrics) + sub_idx
                if value_idx >= len(values):
                    print(f"Insufficient values for day '{day}' in metric '{current_metric}'.")
                    continue
                value = values[value_idx]
                
                # Handle 'Total' submetric
                if submetric.lower() == 'total':
                    full_metric = 'Precipitation (in)'
                else:
                    full_metric = f"{current_metric} ({submetric})"
                
                # Parse the date
                try:
                    date = pd.to_datetime(f"{day} {year}", format='%b %d %Y', errors='coerce')
                except Exception as e:
                    print(f"Error parsing date '{day} {year}': {e}")
                    continue
                
                if pd.isnull(date):
                    print(f"Invalid date '{day} {year}'. Skipping.")
                    continue
                
                data.append({
                    'Date': date,
                    'Metric': full_metric,
                    'Value': value
                })
    
    # Create a long-format DataFrame
    weather_long_df = pd.DataFrame(data)
    
    if weather_long_df.empty:
        print("No valid weather data extracted.")
        return pd.DataFrame()
    
    # Pivot to wide format
    weather_pivot = weather_long_df.pivot_table(
        index='Date',
        columns='Metric',
        values='Value'
    ).reset_index()
    
    # Convert metric columns to numeric types
    for col in weather_pivot.columns:
        if col != 'Date':
            weather_pivot[col] = pd.to_numeric(weather_pivot[col], errors='coerce')
    
    return weather_pivot

def preprocess_weather_files(weather_directory, output_csv='combined_weather_data.csv', num_files=13):
    """
    Processes all weather Excel files, reshapes them, and combines into a single CSV.

    Parameters:
    - weather_directory (str): Path to the directory containing weather Excel files.
    - output_csv (str): Filename for the combined CSV.
    - num_files (int): Number of Excel files to process.

    Returns:
    - None
    """
    all_weather_dfs = []
    start_month = 7  # Assuming file 1.xlsx is July
    start_year = 2023  # Starting year

    for i in range(1, num_files + 1):
        file_name = f"{i}.xlsx"
        file_path = os.path.join(weather_directory, file_name)
        if not os.path.exists(file_path):
            print(f"Weather file '{file_name}' does not exist. Skipping.")
            continue
        
        try:
            # Read the Excel file without headers
            df_weather = pd.read_excel(file_path, header=None)
            print(f"Loaded weather data from '{file_name}'.")
        except Exception as e:
            print(f"Error reading '{file_name}': {e}. Skipping.")
            continue
        
        # Determine the month and year
        month = (start_month + i - 1) % 12 + 1
        year = start_year + (start_month + i - 1) // 12
        
        # Reshape the weather data
        reshaped_df = reshape_weather_data(df_weather, year=year, month=month)
        
        if not reshaped_df.empty:
            all_weather_dfs.append(reshaped_df)
            print(f"Reshaped weather data from '{file_name}' with shape {reshaped_df.shape}.")
        else:
            print(f"Reshaped weather data from '{file_name}' is empty. Skipping.")
    
    if all_weather_dfs:
        combined_weather_df = pd.concat(all_weather_dfs, ignore_index=True)
        combined_weather_df.to_csv(output_csv, index=False)
        print(f"Combined weather data saved to '{output_csv}' with shape {combined_weather_df.shape}.")
    else:
        print("No valid weather data to combine.")

if __name__ == "__main__":
    # Specify the directory containing your weather Excel files
    weather_data_directory = 'ML-Proj-Dataset/Weather/'  # Replace with your actual weather data path
    
    # Preprocess and combine weather data
    preprocess_weather_files(weather_data_directory, output_csv='combined_weather_data.csv', num_files=13)


Loaded weather data from '1.xlsx'.
Missing submetrics for metric 'Jul 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31' at row 1. Skipping.
No valid weather data extracted.
Reshaped weather data from '1.xlsx' is empty. Skipping.
Loaded weather data from '2.xlsx'.
Missing submetrics for metric 'Aug 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31' at row 1. Skipping.
No valid weather data extracted.
Reshaped weather data from '2.xlsx' is empty. Skipping.
Loaded weather data from '3.xlsx'.
Missing submetrics for metric 'Sep 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30' at row 1. Skipping.
No valid weather data extracted.
Reshaped weather data from '3.xlsx' is empty. Skipping.
Loaded weather data from '4.xlsx'.
Missing submetrics for metric 'Oct 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31' at row 1. Skipping.
No valid weather data extracted.
Res

In [31]:
import os
import pandas as pd

def inspect_weather_file(file_path):
    """
    Reads and prints the first few rows of the given Excel weather file.
    """
    try:
        df = pd.read_excel(file_path, header=None)
        print(f"\nContents of '{file_path}':")
        print(df.head(10))  # Print first 10 rows
    except Exception as e:
        print(f"Error reading '{file_path}': {e}")


weather_directory = 'ML-Proj-Dataset/Weather/'  # Replace with your actual path
files = sorted([f for f in os.listdir(weather_directory) if f.endswith('.xlsx')])



# Inspect the first file
first_file = files[0]
file_path = os.path.join(weather_directory, first_file)
inspect_weather_file(file_path)




Contents of 'ML-Proj-Dataset/Weather/1.xlsx':
                                                   0  \
0                                               Time   
1  Jul 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...   

                                                   1  \
0                                   Temperature (°F)   
1  Max Avg Min 97 88.8 81 99 91.5 82 102 91.7 77 ...   

                                                   2  \
0                                     Dew Point (°F)   
1  Max Avg Min 77 74.5 72 79 75.8 75 81 77.0 73 7...   

                                                   3  \
0                                       Humidity (%)   
1  Max Avg Min 74 63.4 47 79 61.3 47 100 63.7 45 ...   

                                                   4  \
0                                   Wind Speed (mph)   
1  Max Avg Min 14 8.9 5 12 6.7 0 35 11.0 0 32 13....   

                                                   5  \
0                                      Pressure (in)

In [49]:
df111 = pd.read_excel('ML-Proj-Dataset/Weather/1.xlsx', header=None)


In [57]:
df111.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2 non-null      object
 1   1       2 non-null      object
 2   2       2 non-null      object
 3   3       2 non-null      object
 4   4       2 non-null      object
 5   5       2 non-null      object
 6   6       2 non-null      object
dtypes: object(7)
memory usage: 244.0+ bytes
